`conda activate gluon`

* https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-quick-start.html
* https://auto.gluon.ai/stable/api/autogluon.timeseries.TimeSeriesDataFrame.html


 - crashes on `RecursiveTabular`

In [1]:
# !pip install autogluon.tabular[all,catboost] -U

In [ ]:
# !pip3 install autogluon -U

In [3]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor


In [4]:
DropCols = ['diff_lag5', 'diff_lag6', 'pct_diff_lag5', 'pct_diff_lag6', 'y_pct_bins',
           'lag1', 'lag2', 'lag3', 'lag4', 'lag5', 'lag6'
            ,"first","start",
            "patent_yearly_total","review_research_ratio_2","review_research_diff_2","pat_div_research"
            ,"y_pct_diff"
           ]

In [5]:
df = pd.read_csv("trends_v6.csv.gz"#,parse_dates=["Year","first","start"]
                ).drop(columns=DropCols,errors="ignore")
df = df.loc[df["year_num"]>=1980]
print(df["variable"].nunique())
df.rename(columns={"y_raw":"target","variable":"item_id","Year":"timestamp"},inplace=True)
df = df[df['item_id'].map(df['item_id'].value_counts()) > 9]

df["timestamp"] = df["timestamp"].apply(pd.Timestamp,freq="Y")
display(df["timestamp"])

df.sort_values("timestamp",inplace=True)
df.reset_index(drop=True)
df

126


/var/folders/hl/bb0z87yd5pzfcp81jc8662kr0000gn/T/ipykernel_38528/1094226677.py:8: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  df["timestamp"] = df["timestamp"].apply(pd.Timestamp,freq="Y")


30     1980-01-01
31     1981-01-01
32     1982-01-01
33     1983-01-01
34     1984-01-01
          ...    
7028   2016-01-01
7029   2017-01-01
7030   2018-01-01
7031   2019-01-01
7032   2020-01-01
Name: timestamp, Length: 4704, dtype: datetime64[ns]

,timestamp,item_id,target,reviews,research_review_diff,research_review_ratio,pct_diff,patent_count,patent_frac,year_num,y_diff
30,1980-01-01,BLAST,103.3293,2.8505,103.3293,1.0000,-0.0017,113.0,4.7221,1980,-0.1794
3666,1980-01-01,glycine,278.6329,5.7009,278.6329,1.0000,0.1071,28.0,1.1701,1980,26.9509
6317,1980-01-01,stem cells,297.8736,14.6086,297.8736,1.0000,0.2108,1.0,0.0418,1980,51.8633
3763,1980-01-01,hippocampus,240.1516,10.3329,220.1984,0.9169,0.0805,0.0,0.0000,1980,17.8916
402,1980-01-01,EEG,705.8463,27.4357,685.8931,0.9717,-0.0196,5.0,0.2089,1980,-14.1061
...,...,...,...,...,...,...,...,...,...,...,...
1372,2020-01-01,Prefrontal cortex,300.5268,20.4289,269.4527,0.8966,-0.1011,3.0,0.0116,2020,-33.8041
4743,2020-01-01,microglia,271.0526,46.2112,198.3206,0.7317,0.0320,5.0,0.0193,2020,8.4148
6822,2020-01-01,viral therapy,1272.5626,267.5452,1242.1653,0.9761,1.4837,21.0,0.0811,2020,760.2058
1008,2020-01-01,Medulla oblongata,27.8744,2.2767,-24.1209,-0.8653,-0.0399,0.0,0.0000,2020,-1.1577


mFil irregular intervals hack
* https://github.com/autogluon/autogluon/issues/2604
    * https://stackoverflow.com/questions/75048264/how-to-work-with-multiple-item-ids-of-varying-length-for-timeseries-predictio

In [6]:
train_data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="item_id",
    timestamp_column="timestamp",
)
print(train_data.shape)
train_data.head()

(4704, 9)


,,target,reviews,research_review_diff,research_review_ratio,pct_diff,patent_count,patent_frac,year_num,y_diff
item_id,timestamp,,,,,,,,,
BLAST,1980-01-01,103.3293,2.8505,103.3293,1.0000,-0.0017,113.0,4.7221,1980,-0.1794
glycine,1980-01-01,278.6329,5.7009,278.6329,1.0000,0.1071,28.0,1.1701,1980,26.9509
stem cells,1980-01-01,297.8736,14.6086,297.8736,1.0000,0.2108,1.0,0.0418,1980,51.8633
hippocampus,1980-01-01,240.1516,10.3329,220.1984,0.9169,0.0805,0.0,0.0000,1980,17.8916
EEG,1980-01-01,705.8463,27.4357,685.8931,0.9717,-0.0196,5.0,0.2089,1980,-14.1061


In [7]:
def forward_fill_missing(ts: TimeSeriesDataFrame, freq="Y") -> TimeSeriesDataFrame:
    original_index = ts.index.get_level_values("timestamp")
    start = original_index[0]
    end = original_index[-1]
    filled_index = pd.date_range(start=start, end=end, freq=freq, name="timestamp")
#     return ts.droplevel("item_id").reindex(filled_index, method="ffill") # orig
    return ts.droplevel("item_id").reindex(filled_index, method=None).fillna(0) # ALT 

train_data = train_data.groupby("item_id").apply(forward_fill_missing)
train_data

target  reviews  research_review_diff  \
item_id    timestamp                                           
BLAST      1980-12-31     0.0      0.0                   0.0   
           1981-12-31     0.0      0.0                   0.0   
           1982-12-31     0.0      0.0                   0.0   
           1983-12-31     0.0      0.0                   0.0   
           1984-12-31     0.0      0.0                   0.0   
...                       ...      ...                   ...   
zebra fish 2015-12-31     0.0      0.0                   0.0   
           2016-12-31     0.0      0.0                   0.0   
           2017-12-31     0.0      0.0                   0.0   
           2018-12-31     0.0      0.0                   0.0   
           2019-12-31     0.0      0.0                   0.0   

                       research_review_ratio  pct_diff  patent_count  \
item_id    timestamp                                                   
BLAST      1980-12-31                    0.0       0.0           0.0   
           1981-12-31                    0.0       0.0           0.0   
           1982-12-31                    0.0       0.0           0.0   
           1983-12-31                    0.0       0.0           0.0   
           1984-12-31                    0.0       0.0           0.0   
...                                      ...       ...           ...   
zebra fish 2015-12-31                    0.0       0.0           0.0   
           2016-12-31                    0.0       0.0           0.0   
           2017-12-31                    0.0       0.0           0.0   
           2018-12-31                    0.0       0.0           0.0   
           2019-12-31                    0.0       0.0           0.0   

                       patent_frac  year_num  y_diff  
item_id    timestamp                                  
BLAST      1980-12-31          0.0       0.0     0.0  
           1981-12-31          0.0       0.0     0.0  
           1982-12-31          0.0       0.0     0.0  
           1983-12-31          0.0       0.0     0.0  
           1984-12-31          0.0       0.0     0.0  
...                            ...       ...     ...  
zebra fish 2015-12-31          0.0       0.0     0.0  
           2016-12-31          0.0       0.0     0.0  
           2017-12-31          0.0       0.0     0.0  
           2018-12-31          0.0       0.0     0.0  
           2019-12-31          0.0       0.0     0.0  

[4585 rows x 9 columns]

* format is giving error
https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-indepth.html#handling-irregular-data-and-missing-values 


* `ValueError: Irregularly-sampled timestamps in this TimeSeriesDataFrame are not compatible with the given frequency 
'Y'`

In [8]:
# train_data = train_data.to_regular_index(freq="Y")
# train_data

I get error:
```
ValueError: Frequency not provided and cannot be inferred. 
This is often due to the time index of the data being 
irregularly sampled. Please ensure that the data set used has a uniform time index, or create the 
`TimeSeriesPredictor` setting `ignore_time_index=True`.
```

In [9]:
# train_data.freq = "Y" ## need to hack it in differently

In [ ]:
predictor = TimeSeriesPredictor(
    prediction_length= 5
    ,path="autogluon-out",
    target="target",
#     eval_metric= "MASE",
    known_covariates_names = ['year_num'],
#     ignore_time_index=True, # try hack 
)

predictor.fit(
    train_data,
    presets="medium_quality",
    time_limit=700,
    excluded_model_types=["RecursiveTabular"],
)

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Setting presets to: medium_quality
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': None,
 'excluded_model_types': ['RecursiveTabular'],
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'medium_quality',
 'num_val_windows': 1,
 'prediction_length': 5,
 'random_seed': None,
 'target': 'target',
 'time_limit': 800,
 'verbosity': 2}
Provided training data set with 4585 rows, 122 items (item = single time series). Average time series length is 37.6. Data frequency is 'A-DEC'.
AutoGluon will save models to autogluon-out/
AutoGluon will gauge predictive performance using evaluation metric: 'mean_wQuantileLoss'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'target'
	known covariates: ['year_num']
	past covariates:  [